<a href="https://colab.research.google.com/github/grinaldo-oliveira/SibGrapi/blob/main/Gera%C3%A7%C3%A3o_de_Gr%C3%A1ficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Doutorado/SibGrapi

/content/drive/MyDrive/Doutorado/SibGrapi


In [3]:
pip install numpy plotly scikit-learn umap-learn


In [ ]:
import os
import numpy as np
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Diretórios onde estão os embeddings
#directories = ["Data/terumo-data-jpeg", "Data/terumo-val"]
directories = ["Data/terumo-val"]

# Cria uma lista de todos os arquivos .npy nos diretórios
npy_files = []
for dir_path in directories:
    for file_name in os.listdir(dir_path):
        if file_name.endswith('.npy'):
            npy_files.append(os.path.join(dir_path, file_name))

# Função para carregar dados
def load_embbed(pickle_path: str) -> tuple:
    data = np.load(pickle_path, allow_pickle=True).tolist()
    file_refs = data['fnames']
    embeds    = data['embeddings']
    labels    = data['labels']
    classes   = data['classes']

    embeds = np.vstack(embeds)
    labels = np.hstack(labels)  # Changed line

    return file_refs, embeds, labels, classes

# Funções para redução de dimensionalidade
def apply_pca(data, n_components):
    return PCA(n_components=n_components).fit_transform(data)

def apply_tsne(data, n_components):
    return TSNE(
        n_components=n_components,
        random_state=42,
        method='barnes_hut',     # Usa o método rápido
        perplexity=30,           # Padrão 30 é ok, mas poderia até diminuir para datasets muito pequenos
        n_iter=500,              # Reduz o número de iterações (padrão é 1000)
        n_jobs=-1,               # Usa todos os CPUs disponíveis (se estiver usando sklearn >= 1.2)
    ).fit_transform(data)

def apply_umap(data, n_components):
    return umap.UMAP(
        n_components=n_components,
        n_neighbors=15,         # Menos vizinhos = mais rápido (padrão é 15, podemos reduzir ainda mais se quiser)
        n_epochs=200,           # Menos épocas = mais rápido (padrão é 200 ou 500 dependendo da versão)
        low_memory=True,        # Otimiza o uso de memória
        random_state=42,        # Para reprodutibilidade
        verbose=False,           # Mostra andamento (opcional, mas ajuda a ver progresso)
    ).fit_transform(data)

# Função para plotar
def plot_embedding(embedding, labels, dims, method_name, file_name):
    title = f"{method_name} - {file_name}"  # <-- aqui agora usa caminho completo
    df = pd.DataFrame()
    df['x'] = embedding[:, 0]
    df['y'] = embedding[:, 1]
    df['label'] = labels.astype(str)  # converter labels para string para usar cores discretas

    if dims == 2:
        fig = px.scatter(df, x='x', y='y', color='label', title=title)
    elif dims == 3:
        df['z'] = embedding[:, 2]
        fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', title=title)

    fig.update_layout(legend_title_text='Label')  # deixa a legenda com nome 'Label'
    fig.show()

# Processamento dos arquivos
for file_path in npy_files:
    print(f"Processando: {file_path}")
    file_refs, embeddings, labels, classes = load_embbed(file_path)
    embeddings.shape, labels.shape

    for method_name, method_func in [('PCA', apply_pca), ('TSNE', apply_tsne), ('UMAP', apply_umap)]:
        # Redução para 2D
        embedding_2d = method_func(embeddings, n_components=2)
        plot_embedding(embedding_2d, labels, dims=2, method_name=method_name, file_name=file_path)

        # Redução para 3D
        embedding_3d = method_func(embeddings, n_components=3)
        plot_embedding(embedding_3d, labels, dims=3, method_name=method_name, file_name=file_path)

Processando: Data/terumo-val/Phikon.npy


In [ ]:
file_path

'Data/terumo-data-jpeg/Phikon.npy'